# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse      Shipments  \
0  Jan  4 2023  1:56PM  254290        10      MSP217225   
1  Jan  4 2023  1:56PM  254290        10      MSP217210   
2  Jan  4 2023  1:56PM  254289        10   SONSB0002070   
3  Jan  4 2023  1:15PM  254288        10  Yusin-8989598   
4  Jan  4 2023  1:01PM  254286        19      MSP217219   
5  Jan  4 2023 12:55PM  254284        12         HH8661   
6  Jan  4 2023 12:47PM  254282        19    ADV80010028   
7  Jan  4 2023 12:47PM  254282        19    ADV80010029   
8  Jan  4 2023 12:47PM  254282        19    ADV80010030   
9  Jan  4 2023 12:47PM  254282        19    ADV80010031   

                        Customer ShipmentStatus  
0               Methapharm, Inc.       Released  
1                   Methapharm-G       Released  
2  Nextsource Biotechnology, LLC       Released  
3              Yusen – 3D Matrix       Released  
4               Methapharm, Inc.       Released  
5                      Hush Hush      Executing  
6            AdvaGen Pharma, Ltd       Released  
7            AdvaGen Pharma, Ltd       Released  
8            AdvaGen Pharma, Ltd       Released  
9            AdvaGen Pharma, Ltd       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
22  254284      Executing          1
23  254286       Released          1
24  254288       Released          1
25  254289       Released          1
26  254290       Released          2

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
254284                NaN        1.0       NaN
254286                NaN        NaN       1.0
254288                NaN        NaN       1.0
254289                NaN        NaN       1.0
254290                NaN        NaN       2.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
254195                0.0        6.0       5.0
254196                0.0        0.0      17.0
254198                0.0        4.0      22.0
254199                0.0        0.0       1.0
254206                1.0        0.0      16.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
254195                  0          6         5
254196                  0          0        17
254198                  0          4        22
254199                  0          0         1
254206                  1          0        16

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               254195          0          6         5
1               254196          0          0        17
2               254198          0          4        22
3               254199          0          0         1
4               254206          1          0        16

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               254195                   6        5
1               254196                           17
2               254198                   4       22
3               254199                            1
4               254206         1                 16

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Jan  4 2023  1:56PM  254290        10                Methapharm, Inc.
1   Jan  4 2023  1:56PM  254290        10                    Methapharm-G
2   Jan  4 2023  1:56PM  254289        10   Nextsource Biotechnology, LLC
3   Jan  4 2023  1:15PM  254288        10               Yusen – 3D Matrix
4   Jan  4 2023  1:01PM  254286        19                Methapharm, Inc.
5   Jan  4 2023 12:55PM  254284        12                       Hush Hush
6   Jan  4 2023 12:47PM  254282        19             AdvaGen Pharma, Ltd
20  Jan  4 2023 12:39PM  254278        16              Innova Softgel LLC
21  Jan  4 2023 12:39PM  254277        18              Innova Softgel LLC
22  Jan  4 2023 11:46AM  254275        19  Avet Pharmaceuticals Labs, Inc

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Jan  4 2023  1:56PM  254290        10                Methapharm, Inc.   
1  Jan  4 2023  1:56PM  254290        10                    Methapharm-G   
2  Jan  4 2023  1:56PM  254289        10   Nextsource Biotechnology, LLC   
3  Jan  4 2023  1:15PM  254288        10               Yusen – 3D Matrix   
4  Jan  4 2023  1:01PM  254286        19                Methapharm, Inc.   
5  Jan  4 2023 12:55PM  254284        12                       Hush Hush   
6  Jan  4 2023 12:47PM  254282        19             AdvaGen Pharma, Ltd   
7  Jan  4 2023 12:39PM  254278        16              Innova Softgel LLC   
8  Jan  4 2023 12:39PM  254277        18              Innova Softgel LLC   
9  Jan  4 2023 11:46AM  254275        19  Avet Pharmaceuticals Labs, Inc   

  Completed Executing Released  
0                            2  
1                            2  
2                            1  
3                            1  
4                            1  
5                   1           
6                           14  
7                            1  
8                            1  
9                   1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Jan  4 2023  1:56PM  254290        10                Methapharm, Inc.   
1  Jan  4 2023  1:56PM  254290        10                    Methapharm-G   
2  Jan  4 2023  1:56PM  254289        10   Nextsource Biotechnology, LLC   
3  Jan  4 2023  1:15PM  254288        10               Yusen – 3D Matrix   
4  Jan  4 2023  1:01PM  254286        19                Methapharm, Inc.   
5  Jan  4 2023 12:55PM  254284        12                       Hush Hush   
6  Jan  4 2023 12:47PM  254282        19             AdvaGen Pharma, Ltd   
7  Jan  4 2023 12:39PM  254278        16              Innova Softgel LLC   
8  Jan  4 2023 12:39PM  254277        18              Innova Softgel LLC   
9  Jan  4 2023 11:46AM  254275        19  Avet Pharmaceuticals Labs, Inc   

  Released Executing Completed  
0        2                      
1        2                      
2        1                      
3        1                      
4        1                      
5                  1            
6       14                      
7        1                      
8        1                      
9                  1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Jan  4 2023  1:56PM  254290        10               Methapharm, Inc.   
1  Jan  4 2023  1:56PM  254290        10                   Methapharm-G   
2  Jan  4 2023  1:56PM  254289        10  Nextsource Biotechnology, LLC   
3  Jan  4 2023  1:15PM  254288        10              Yusen – 3D Matrix   
4  Jan  4 2023  1:01PM  254286        19               Methapharm, Inc.   

  Released Executing Completed  
0        2                      
1        2                      
2        1                      
3        1                      
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Jan  4 2023  1:56PM  254290        10               Methapharm, Inc.   
1  Jan  4 2023  1:56PM  254290        10                   Methapharm-G   
2  Jan  4 2023  1:56PM  254289        10  Nextsource Biotechnology, LLC   
3  Jan  4 2023  1:15PM  254288        10              Yusen – 3D Matrix   
4  Jan  4 2023  1:01PM  254286        19               Methapharm, Inc.   

   Released  Executing  Completed  
0       2.0        NaN        NaN  
1       2.0        NaN        NaN  
2       1.0        NaN        NaN  
3       1.0        NaN        NaN  
4       1.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Jan  4 2023  1:56PM  254290        10               Methapharm, Inc.   
1  Jan  4 2023  1:56PM  254290        10                   Methapharm-G   
2  Jan  4 2023  1:56PM  254289        10  Nextsource Biotechnology, LLC   
3  Jan  4 2023  1:15PM  254288        10              Yusen – 3D Matrix   
4  Jan  4 2023  1:01PM  254286        19               Methapharm, Inc.   

   Released  Executing  Completed  
0       2.0        0.0        0.0  
1       2.0        0.0        0.0  
2       1.0        0.0        0.0  
3       1.0        0.0        0.0  
4       1.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1525602      24.0        0.0        0.0
12          254284       0.0        1.0        0.0
15         1016966      65.0       34.0       13.0
16          508543       2.0        0.0        0.0
18          254277       1.0        0.0        0.0
19         2033926      55.0        6.0        1.0
21          254266       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1525602      24.0        0.0        0.0
1        12   254284       0.0        1.0        0.0
2        15  1016966      65.0       34.0       13.0
3        16   508543       2.0        0.0        0.0
4        18   254277       1.0        0.0        0.0
5        19  2033926      55.0        6.0        1.0
6        21   254266       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      24.0        0.0        0.0
1        12       0.0        1.0        0.0
2        15      65.0       34.0       13.0
3        16       2.0        0.0        0.0
4        18       1.0        0.0        0.0
5        19      55.0        6.0        1.0
6        21       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   24.0
1        12  Released    0.0
2        15  Released   65.0
3        16  Released    2.0
4        18  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15   16   18    19   21
Status                                         
Completed   0.0  0.0  13.0  0.0  0.0   1.0  0.0
Executing   0.0  1.0  34.0  0.0  0.0   6.0  0.0
Released   24.0  0.0  65.0  2.0  1.0  55.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15   16   18    19   21
0          Completed   0.0  0.0  13.0  0.0  0.0   1.0  0.0
1          Executing   0.0  1.0  34.0  0.0  0.0   6.0  0.0
2           Released  24.0  0.0  65.0  2.0  1.0  55.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15   16   18    19   21
0  Completed   0.0  0.0  13.0  0.0  0.0   1.0  0.0
1  Executing   0.0  1.0  34.0  0.0  0.0   6.0  0.0
2   Released  24.0  0.0  65.0  2.0  1.0  55.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()